# **Donut 🍩 : Document Understanding Transformer**
Donut 🍩, Document understanding transformer, is a new method of document understanding that utilizes an **OCR-free** end-to-end Transformer model. Donut does not require off-the-shelf OCR engines/APIs, yet it shows state-of-the-art performances on various visual document understanding tasks, such as visual document classification or information extraction (a.k.a. document parsing). In addition, we present SynthDoG 🐶, Synthetic Document Generator, that helps the model pre-training to be flexible on various languages and domains.

### **Setting**

In [ ]:
!pip install transformers==4.25.1
!pip install pytorch-lightning==1.6.4
!pip install timm==0.5.4
!pip install gradio
!pip install donut-python

In [ ]:
import argparse
import gradio as gr
import torch
from PIL import Image

from donut import DonutModel

In [ ]:
def demo_process_vqa(input_img, question):
    global pretrained_model, task_prompt, task_name
    input_img = Image.fromarray(input_img)
    user_prompt = task_prompt.replace("{user_input}", question)
    output = pretrained_model.inference(input_img, prompt=user_prompt)["predictions"][0]
    return output


def demo_process(input_img):
    global pretrained_model, task_prompt, task_name
    input_img = Image.fromarray(input_img)
    output = pretrained_model.inference(image=input_img, prompt=task_prompt)["predictions"][0]
    return output

### **Document Classification**

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--task", type=str, default="rvlcdip")
parser.add_argument("--pretrained_path", type=str, default="naver-clova-ix/donut-base-finetuned-rvlcdip")
args, left_argv = parser.parse_known_args()

task_name = args.task
if "docvqa" == task_name:
    task_prompt = "<s_docvqa><s_question>{user_input}</s_question><s_answer>"
else:
    task_prompt = f"<s_{task_name}>"

pretrained_model = DonutModel.from_pretrained(args.pretrained_path)

if torch.cuda.is_available():
    pretrained_model.half()
    device = torch.device("cuda")
    pretrained_model.to(device)
else:
    pretrained_model.encoder.to(torch.bfloat16)

pretrained_model.eval()

demo = gr.Interface(
    fn=demo_process_vqa if task_name == "docvqa" else demo_process,
    inputs=["image", "text"] if task_name == "docvqa" else "image",
    outputs="json",
    title=f"Donut 🍩 demonstration for `{task_name}` task",
)
demo.launch()

### **Document VQA**

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--task", type=str, default="docvqa")
parser.add_argument("--pretrained_path", type=str, default="naver-clova-ix/donut-base-finetuned-docvqa")
args, left_argv = parser.parse_known_args()

task_name = args.task
if "docvqa" == task_name:
    task_prompt = "<s_docvqa><s_question>{user_input}</s_question><s_answer>"
else:
    task_prompt = f"<s_{task_name}>"

pretrained_model = DonutModel.from_pretrained(args.pretrained_path)

if torch.cuda.is_available():
    pretrained_model.half()
    device = torch.device("cuda")
    pretrained_model.to(device)
else:
    pretrained_model.encoder.to(torch.bfloat16)

pretrained_model.eval()

demo = gr.Interface(
    fn=demo_process_vqa if task_name == "docvqa" else demo_process,
    inputs=["image", "text"] if task_name == "docvqa" else "image",
    outputs="json",
    title=f"Donut 🍩 demonstration for `{task_name}` task",
)
demo.launch()

# **DiffSTE : Diffusion models for Scene Text Editing**
edit scene text into different font styles and colors following given text instruction. Specifically, we propose to improve pre-trained diffusion models with a dual encoder design, which includes a character encoder for better text legibility and an instruction encoder for better style control. We then utilize an instruction tuning framework to train our model learn the mapping from the text instruction to the corresponding image with either the specified style or the style of the surrounding texts in the background. Such a training method further brings our model the zero-shot generalization ability to the following three scenarios: generating text with unseen font variation, e.g. italic and bold, mixing different fonts to construct a new font, and using more relaxed forms of natural language as the instructions to guide the generation task.



### **Setting**

In [16]:
# coord -> http://maschek.hu/imagemap/imgmap/
coord = "117,108,233,144"

In [ ]:
# character masking
import cv2
import numpy as np

img = '/content/DocumentAI_OCR/samsung.png'
image = cv2.imread(img)
filename = img.split('/'[-1])

temp = tuple(map(int, coord.split(',')))
x, y, w, h = temp

image2 = np.zeros((image.shape[0], image.shape[1]), dtype="uint8")
cv2.rectangle(image2, (x, y), (w, h), 255, -1)
cv2.imwrite(f'/content/masked_testimg.png', image2)

### **Scene Text Editing**

In [1]:
!git clone https://github.com/UCSB-NLP-Chang/DiffSTE.git
%cd DiffSTE

Cloning into 'DiffSTE'...
remote: Enumerating objects: 251, done.
remote: Counting objects: 100% (251/251), done.
remote: Compressing objects: 100% (196/196), done.
remote: Total 251 (delta 52), reused 246 (delta 47), pack-reused 0
Receiving objects: 100% (251/251), 6.57 MiB | 22.82 MiB/s, done.
Resolving deltas: 100% (52/52), done.
/content/DiffSTE


In [ ]:
# requirements.txt -> flax==0.7.2

In [ ]:
!curl -sSL https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -o miniconda.sh
!bash ./miniconda.sh -bfp /usr/local
!conda --version
!conda create --name DiffSTE python=3.8 -y

In [ ]:
# pretrained model download
!wget --load-cookies ~/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies ~/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1fc0RKGWo6MPSJIZNIA_UweTOPai64S9f' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1fc0RKGWo6MPSJIZNIA_UweTOPai64S9f" -O diffste.ckpt && rm -rf ~/cookies.txt

In [ ]:
# style-free generation
%%bash
source activate DiffSTE
pip install -r requirements.txt
pip install jax --upgrade

python generate.py \
    --ckpt_path /content/DiffSTE/diffste.ckpt\
    --in_image /content/DocumentAI_OCR/samsung.png \
    --in_mask /content/masked_testimg.png \
    --text DASU \
    --out_dir /content/

In [ ]:
# style-conditional generation
%%bash
source activate DiffSTE
pip install -r requirements.txt
pip install jax --upgrade

python generate.py \
    --ckpt_path /content/DiffSTE/diffste.ckpt \
    --in_image /content/DocumentAI_OCR/OMG.png \
    --in_mask /content/DocumentAI_OCR/masked3.png \
    --text QnA \
    --font Caprasimo \
    --color white \
    --out_dir /content/

# **EasyOCR**
Ready-to-use OCR with 80+ supported languages and all popular writing scripts including: Latin, Chinese, Arabic, Devanagari, Cyrillic, etc.
- opensource text detection & recognition model
- support language -> https://www.jaided.ai/easyocr/

### **Setting**

In [ ]:
!pip install easyocr

In [ ]:
!unzip DocumentAI.zip

### **Detection & Crop**

In [ ]:
def crop_box(ocr):
    max_diff = 0
    max_row = []
    for i in ocr:
        diff = i[0][2][1]-i[0][0][1]
        x_diff = i[0][2][0]-i[0][0][0]
        if diff > max_diff and x_diff > 0:
            max_diff = diff
            max_row = i[0]
    box = tuple(max_row[0] + max_row[2])
    return box

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

def img_show(img):
    img = np.array(img)
    plt.imshow(img)

### **English OCR**

In [ ]:
import easyocr

en_imgpath = "/content/DocumentAI/Bumblebee.jpg"
en_img = Image.open(en_imgpath)
img_show(en_img)

In [ ]:
en_reader = easyocr.Reader(['en'])
en_ocr = en_reader.readtext(en_imgpath)
box = crop_box(en_ocr)
text = en_img.crop(box)
img_show(text)

In [ ]:
# recognition
print(en_ocr)

### **Korean OCR**

In [ ]:
ko_imgpath = "/content/DocumentAI/Busanhaeng.jpg"
ko_img = Image.open(ko_imgpath)
img_show(ko_img)

In [ ]:
ko_reader = easyocr.Reader(['ko'])
ko_ocr = ko_reader.readtext(ko_imgpath)
box = crop_box(ko_ocr)
text = ko_img.crop(box)
img_show(text)

In [ ]:
# recognition
print(ko_ocr)